# Installing dependencies
Run it once

In [ ]:
!pip install tensorflow
!pip install gym[all]
!pip install keras 
!pip install keras-rl2==1.0.5

# Test random enviroment with OpenAI Gym

In [1]:
import gym
import random 

In [2]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [3]:
actions

2

In [4]:
episodes = 10 
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
        
    print('Episodes:{} Score:{}'.format(episode, score))
env.close()

Episodes:1 Score:11.0
Episodes:2 Score:24.0
Episodes:3 Score:27.0
Episodes:4 Score:24.0
Episodes:5 Score:27.0
Episodes:6 Score:21.0
Episodes:7 Score:17.0
Episodes:8 Score:18.0
Episodes:9 Score:15.0
Episodes:10 Score:11.0


# Create a deep learning model with keras

In [19]:
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [38]:
def build_model(states, actions):
    model = tensorflow.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [33]:
del model

In [41]:
model = build_model(states, actions)

In [42]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 4)                 0         
                                                                 
 dense_12 (Dense)            (None, 24)                120       
                                                                 
 dense_13 (Dense)            (None, 24)                600       
                                                                 
 dense_14 (Dense)            (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# Build agent with keras-rl

In [43]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [44]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=5000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [45]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 154s 15ms/step - reward: 1.0000
104 episodes - episode_reward: 94.577 [9.000, 200.000] - loss: 1.826 - mae: 17.983 - mean_q: 36.226

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 151s 15ms/step - reward: 1.0000
60 episodes - episode_reward: 168.500 [18.000, 200.000] - loss: 2.288 - mae: 33.787 - mean_q: 67.806

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 143s 14ms/step - reward: 1.0000
52 episodes - episode_reward: 192.750 [52.000, 200.000] - loss: 4.950 - mae: 40.910 - mean_q: 82.087

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 136s 14ms/step - reward: 1.0000
52 episodes - episode_reward: 192.750 [74.000, 200.000] - loss: 16.731 - mae: 45.134 - mean_q: 90.121

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 148s 15ms/step - reward: 1.00

In [46]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [47]:
_ = dqn.test(env, nb_episodes=15, visualize=True)
env.close()

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


# Reloding agent from memory

In [49]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [50]:
del model
del dqn 
del env

In [54]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [55]:
dqn.load_weights('dqn_weights.h5f')

In [56]:
_ = dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
